# 📘 Register and Use Unity Catalog Tool with LangChain Agent
This notebook demonstrates how to:
- Define a Python function
- Register it using SQL in Unity Catalog
- Load it with `UCFunctionToolkit`
- Use it with a LangChain agent powered by a Databricks-hosted LLM

---

## ⚙️ Tool Setup

We defined a Python function to extract the top 10 keywords from a text string and registered it as a Unity Catalog Python function:

```sql
CREATE FUNCTION IF NOT EXISTS arao.tools.extract_keywords(text STRING)
RETURNS ARRAY<STRING>
LANGUAGE PYTHON
AS $$
import re, collections
stop = {"the", "and", "of", "in", "to", "a", "an", "is"}
tokens = re.findall(r"\b\w+\b", text.lower())
return [w for w, _ in collections.Counter(
    t for t in tokens if t not in stop
).most_common(10)]
$$;
```

This function is discoverable using `UCFunctionToolkit` and can be loaded into LangChain as a tool.

## 🔧 Step 1: Register the Function via SQL
Register a Python function in Unity Catalog using SQL. Update `arao.tools` to your catalog.schema as needed.

## 🔧 Step 1: Register the Function in Unity Catalog

In [0]:
%sql
CREATE FUNCTION IF NOT EXISTS arao.tools.extract_keywords(text STRING)
RETURNS ARRAY<STRING>
LANGUAGE PYTHON
AS $$
import re, collections
stop = {"the", "and", "of", "in", "to", "a", "an", "is"}
tokens = re.findall(r"\b\w+\b", text.lower())
return [w for w, _ in collections.Counter(
    t for t in tokens if t not in stop
).most_common(10)]
$$;


## 🧪 Step 2: Test the Function in SQL

In [0]:
%sql
SELECT arao.tools.extract_keywords(
  'Databricks unifies data, engineering, analytics, and AI workflows into a single platform.'
);


## 🧠 Step 3: Load the Function as a Tool in LangChain

In [0]:
%pip install --upgrade databricks-langchain langchain-community langchain databricks-sql-connector
dbutils.library.restartPython()

---

## 🛠️ Tool Wrapping

Because the registered UC function expects a named parameter (`text`), we wrapped it like this:

```python
from langchain.tools import Tool
from databricks_langchain import UCFunctionToolkit

toolkit = UCFunctionToolkit(function_names=["arao.tools.extract_keywords"])
uc_tool = toolkit.tools[0]

tools = [
    Tool(
        name=uc_tool.name,
        func=lambda text: uc_tool.run({"text": text}),
        description=uc_tool.description
    )
]
```


---

## 🔎 Model Limitation: No Function Calling

We verified that `databricks-llama-4-maverick` does **not support** OpenAI-style tool calling via JSON schema (i.e., the `"functions"` field).  
Attempting to use `AgentType.OPENAI_FUNCTIONS` returns a 400 error.

One the cells below has code to identify this


## 🛠️ Step 3: Load the UC Tool in Python

In [0]:
from databricks_langchain import ChatDatabricks, UCFunctionToolkit

# Replace with your actual model endpoint name
llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", temperature=0.2)

from langchain.tools import Tool
from databricks_langchain import UCFunctionToolkit

# Load the UC function
toolkit = UCFunctionToolkit(function_names=["arao.tools.extract_keywords"])
uc_tool = toolkit.tools[0]  # assuming only one tool

# Wrap it to ensure correct input mapping
tools = [
    Tool(
        name=uc_tool.name,
        func=lambda text: uc_tool.run({"text": text}),
        description=uc_tool.description
    )
]


## 🧩 Step 4: Create a LangChain Agent and Use the Tool
## 🧠 Step 5: Initialize the LangChain Agent (ReAct Style)


We use `AgentType.ZERO_SHOT_REACT_DESCRIPTION`, which works via ReAct-style text prompts and is compatible with non-structured models:

```python
from databricks_langchain import ChatDatabricks
from langchain.agents import initialize_agent, AgentType

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick", temperature=0.2)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```
---

In [0]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # ✅ fallback agent type
    verbose=True
)



---

## 🧪 Example Query

```python
response = agent.run("Extract keywords from: 'Databricks simplifies AI workflows across enterprises.'")
print(response)
```

**Output:**
```
['databricks', 'simplifies', 'ai', 'workflows', 'across', 'enterprises']
```

In [0]:
response = agent.run("Extract keywords from: 'Databricks simplifies AI workflows across enterprises.'")
print(response)

---

## ✅ Summary

| Component | Value |
|----------|-------|
| LLM | `databricks-llama-4-maverick` |
| Tool Name | `arao.tools.extract_keywords` |
| Agent Type | `ZERO_SHOT_REACT_DESCRIPTION` |
| Function Calling | ❌ Not supported |
| Status | 🟢 Working end-to-end |

---

The Below cell helps identify the function calling style

In [0]:
import requests
import os
import json

# Replace with your model serving endpoint
endpoint_name = "databricks-llama-4-maverick"

# Build the serving endpoint URL
host = spark.conf.get("spark.databricks.workspaceUrl")
url = f"https://{host}/serving-endpoints/{endpoint_name}/invocations"

# Get your notebook's current token automatically
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Create a test OpenAI-style function-calling payload
payload = {
    "messages": [
        {"role": "user", "content": "What's the weather in San Francisco?"},
    ],
    "functions": [
        {
            "name": "get_weather",
            "description": "Get weather info",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"}
                },
                "required": ["location"]
            }
        }
    ],
    "function_call": "auto"
}

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Make the request and interpret the result
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    print("✅ This model supports OpenAI-style function calling.")
else:
    error_text = response.text
    if "unknown field" in error_text and "functions" in error_text:
        print("❌ This model does NOT support OpenAI-style function calling.")
    else:
        print("⚠️ Unexpected error:")
        print(error_text)

✅ Done! You have now registered, loaded, and used a Unity Catalog tool in a generative AI workflow.